# Webhooks

> Prerequisites: Complete Notebooks 1-2 and have your MANUS_API_KEY configured.

In our previous notebooks, we used polling to check the status of our tasks. While simple, this "pull" method of repeatedly asking "Are you done yet?" is inefficient and doesn't scale for real-world applications. It creates network noise, introduces delays, and wastes resources.

This notebook introduces webhooks, a powerful, event-driven "push" model. Instead of you asking for updates, Manus tells you the moment a task is complete by sending a notification directly to your application. This is the professional standard for building responsive, scalable, and efficient automations.

To accomplish this, we'll use two modern tools: FastAPI to create a simple and fast web server, and Modal to deploy it instantly to a live, public URL without any complex setup.

In this notebook, you will:

1. Deploy a serverless webhook receiver using Modal and FastAPI.
2. Process message_ended events to know exactly when a task is finished.
3. Use the event payload to retrieve the full conversation history from the API.

By the end, you'll have a production-ready foundation for event-driven automation, which will be the core of the intelligent support agent we will build in our final section.

## Why Webhooks Win at Scale

A "push" model is where the server initiates communication. Instead of your application constantly asking for updates, the server pushes information to you the moment it becomes available. 

Webhooks use this model, giving you three major advantages for building professional applications.

1. Instantaneous Response : Your application is notified the moment an event occurs. This allows your workflows to react in real-time without any delays.
2. Effeciency : Webhooks eliminate constant, repetitive API calls. This drastically reduces network traffic and saves server resources.
3. Built for Scale : The webhook model handles a massive volume of events with minimal overhead. Your application can grow without a change in architecture.

## Implementing our First Webhook

To start, we'll create our webhook server. We'll use FastAPI to build the web application and Modal to serve it from a temporary, cloud-hosted environment. This gives us a public URL for testing without needing ngrok or a full deployment.

Create a new file named `server.py` and add the following code:

```py
import modal

app = modal.App("manus-webhook-receiver")

image = modal.Image.debian_slim().pip_install("fastapi[standard]")

@app.function(image=image)
@modal.fastapi_endpoint(method="POST")
def handle_webhook(payload: dict):
    print("🔔 Received webhook event:")
    print(payload)
    
    return {"status": "received"}

```

This code does three key things:

1, Defines a Modal Stub which describes our cloud application.
2. Creates a standard FastAPI app with a /webhook endpoint that accepts POST requests.
3. Attaches the FastAPI app to our Modal Stub, making it servable.


### Run Your Development Server

Now, run the server using the modal serve command. This command uploads your code to a temporary container on Modal and streams all logs directly back to your terminal in real-time.

```bash
✓ Created objects.
├── 🔨 Created mount /Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/server.py
└── 🔨 Created web function handle_webhook => https://ivanleo-manus--manus-webhook-receiver-handle-webhook-dev.modal.run
️️⚡️ Serving... hit Ctrl-C to stop!
└── Watching /Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop.
```

After a moment, Modal will output a temporary public URL. This URL is live as long as the command is running, in the example above, my temporary public url was https://ivanleo-manus--manus-webhook-receiver-serve-dev.modal.run

Your server is now live and waiting for events. Any request sent to this URL will be handled by your code, and any print statements will appear directly in your terminal. To stop the server, simply press Ctrl+C.

### Testing Our Webhook

Now that our server is running with modal serve, let's send it a test request to confirm that we can hit our server endpoint without any issues.

In [1]:
import requests
import json

MODAL_WEBHOOK_URL = "https://ivanleo-manus--manus-webhook-receiver-handle-webhook-dev.modal.run"

# Create a test payload that mimics a Manus webhook
test_payload = {
    "event_type": "message_ended",
    "payload": {
        "id": "test-task-123",
        "status": "completed"
    }
}

# Send the test request
response = requests.post(
    MODAL_WEBHOOK_URL,
    headers={"Content-Type": "application/json"},
    json=test_payload
)

print(f"Response Status: {response.status_code}")
print(f"Response Body: {response.json()}")

/Users/ivanleo-work/Documents/coding/manus-api-webhook-workshop/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Response Status: 200
Response Body: {'status': 'received'}


## Registering Your Webhook

Now, let's tell Manus where to send real-time events. This involves two steps: registering your webhook URL with the Manus API and then creating a task to trigger an event.

In [3]:
from lib.env import get_env_key, get_base_url

# Load your API configuration
API_KEY = get_env_key()
BASE_URL = get_base_url()

# Register the webhook with Manus
webhook_url = "https://ivanleo-manus--manus-webhook-receiver-handle-webhook-dev.modal.run"

response = requests.post(
    f"{BASE_URL}/webhooks",
    headers={
        "API_KEY": API_KEY,
        "Content-Type": "application/json"
    },
    json={"webhook": {"url": webhook_url}}
)

webhook_data = response.json()
webhook_id = webhook_data["webhook_id"]

print(f"✓ Webhook registered successfully!")
print(f"Webhook ID: {webhook_id}")
print(f"Endpoint: {webhook_url}")

✓ Webhook registered successfully!
Webhook ID: UThoZXKL3nER8oCJ7GHEaR
Endpoint: https://ivanleo-manus--manus-webhook-receiver-handle-webhook-dev.modal.run


Now let's create a simple task. When it completes, Manus will automatically send a message_ended event to our Modal server.

In [4]:
# Create a simple task that will complete quickly
task_data = {
    "prompt": "What is 2 + 2? Please provide a brief answer.",
    "agentProfile": "manus-1.5",
    "taskMode": "agent"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers={
        "API_KEY": API_KEY,
        "Content-Type": "application/json"
    },
    json=task_data
)

task = response.json()
print(f"✓ Task created successfully!")
print(f"Task ID: {task['task_id']}")
print(f"Task URL: {task['task_url']}")
print(f"\n⏳ Check your Modal terminal - you should see the webhook event arrive shortly!")

✓ Task created successfully!
Task ID: EFyyM466LvHRFtJRHvhHBN
Task URL: https://manus.im/app/EFyyM466LvHRFtJRHvhHBN

⏳ Check your Modal terminal - you should see the webhook event arrive shortly!


 Within a few seconds, you should see the real webhook event from Manus

```
🔔 Received webhook event:
{'event_id': 'task_created_EFyyM466LvHRFtJRHvhHBN_1763506680', 'event_type': 'task_created', 'task_detail': {'task_id': 'EFyyM466LvHRFtJRHvhHBN', 'task_title': 'What is 2 + 2?', 'task_url': 'https://manus.im/app/EFyyM466LvHRFtJRHvhHBN'}}
   POST / -> 200 OK  (duration: 3.40 s, execution: 135.1 ms)

🔔 Received webhook event:
{'event_id': 'task_stopped_EFyyM466LvHRFtJRHvhHBN_1763506696', 'event_type': 'task_stopped', 'task_detail': {'task_id': 'EFyyM466LvHRFtJRHvhHBN', 'task_title': 'What is 2 + 2?', 'task_url': 'https://manus.im/app/EFyyM466LvHRFtJRHvhHBN', 'message': '**2 + 2 = 4**', 'stop_reason': 'finish'}}
```

You have now successfully deployed a webhook receiver with Modal, registered it with the Manus API, and received a real-time event triggered by a new task. This confirms the entire event-driven workflow is operational, from task creation to event notification, all without relying on polling.

## Processing Events

Receiving a webhook notification is the first step. The real power comes from what you do with it. The task_stopped event gives us a summary, but often we need the entire conversation history, including all messages and any files the agent generated.

To call the Manus API from our Modal function, we need our API key. We should never hardcode secrets in our code. Modal provides a secure way to handle this using modal.Secret.

Run the following command in your terminal. This creates a secret named manus-api-key that our script can securely access.

```bash
modal secret create manus-api-key MANUS_API_KEY="<YOUR_MANUS_API_KEY_HERE>"
```

This give sthe following output

```bash
(manus-api-webhook-workshop) ~/Documents/coding/manus-api-webhook-workshop ivan $ (main) modal secret create manus-api-key MANUS_API_KEY="sk-asd123213"
Created a new secret 'manus-api-key' with the key 'MANUS_API_KEY'

Use it in your Modal app:

                                                                                                                                                                                                                            
@app.function(secrets=[modal.Secret.from_name("manus-api-key")])                                                                                                                                                            
def some_function():                                                                                                                                                                                                        
    os.getenv("MANUS_API_KEY")                                                                                                                                                                                              
                                                                                                                                                                                                                            
(manus-api-webhook-workshop) ~/Documents/coding/manus-api-webhook-workshop ivan $ (main) 
```

### Fetching Conversation History

Now, let's modify server.py to use this secret and fetch the task details. We'll add the requests library to our image and update our webhook handler.

```py
import modal
import os
import requests

# Define our Modal app and image
app = modal.App("manus-webhook-receiver")
image = modal.Image.debian_slim().pip_install("fastapi[standard]", "requests")

# Define the secret we created
manus_secret = modal.Secret.from_name("manus-api-key")

@app.function(image=image, secrets=[manus_secret])
@modal.fastapi_endpoint(method="POST")
def handle_webhook(payload: dict):
    """
    Receives webhook events, and if a task has stopped, fetches its full details.
    """
    event_type = payload.get("event_type")
    print(f"🔔 Received event: {event_type}")

    # Check if the task has finished
    if event_type == "task_stopped":
        task_detail = payload.get("task_detail", {})
        task_id = task_detail.get("task_id")

        if task_id:
            print(f"✅ Task {task_id} stopped. Fetching full details...")
            
            # Fetch the full task object from the Manus API
            api_key = os.environ["MANUS_API_KEY"]
            base_url = "https://api.manus.ai/v1"
            
            response = requests.get(
                f"{base_url}/tasks/{task_id}",
                headers={"API_KEY": api_key}
            )
            
            if response.status_code == 200:
                full_task_details = response.json()
                print("🔍 Full task details:")
                print(full_task_details)
            else:
                print(f"❌ Error fetching task details: {response.status_code}")

    return {"status": "received"}
```

Let's restart our webapp and see what we get when we create a task below

In [5]:
# Create a new task to trigger our updated webhook
task_data = {
    "prompt": "Tell me what's 2+2 and write your answer in an answer.md file",
    "agentProfile": "manus-1.5",
    "taskMode": "agent"
}

response = requests.post(
    f"{BASE_URL}/tasks",
    headers={"API_KEY": API_KEY},
    json=task_data
)

task = response.json()
print(f"✓ Task created: {task['task_id']}")
print(f"\n⏳ Watch your 'modal serve' terminal for the full task details!")

✓ Task created: 9rP8DLRXVC9fkKBKEELXkg

⏳ Watch your 'modal serve' terminal for the full task details!


Now, check your modal serve logs. When the task completes, you will see your new log messages, followed by the complete JSON object for the task, including the full conversation history in the output array.

```bash
🔔 Received event: task_created
   POST / -> 200 OK  (duration: 4.17 s, execution: 144.0 ms)
🔔 Received event: task_stopped
✅ Task 9rP8DLRXVC9fkKBKEELXkg stopped. Fetching full details...
🔍 Full task details:
# {
#   "id": "9rP8DLRXVC9fkKBKEELXkg",
#   "object": "task",
#   "created_at": "1763507071",
#   "updated_at": "1763507100",
#   "status": "completed",
#   "model": "manus-1.5-agent",
#   "metadata": {
#     "task_url": "https://manus.im/app/9rP8DLRXVC9fkKBKEELXkg"
#   },
#   "output": [
#     {
#       "id": "wP6Re2TjYi18kx7Crdbnlc",
#       "status": "completed",
#       "role": "user",
#       "type": "message",
#       "content": [
#         {
#           "type": "output_text",
#           "text": "Tell me what's 2+2 and write your answer in an answer.md file"
#         }
#       ]
#     },
#     {
#       "id": "CNB92PkeS94q8Cc6LmmcYW",
#       "status": "completed",
#       "role": "assistant",
#       "type": "message",
#       "content": [
#         {
#           "type": "output_text",
#           "text": "Got it! I'll calculate 2+2 and save the answer to an answer.md file for you."
#         }
#       ]
#     },
#     {
#       "id": "l3jxHYsaaIOUKP25D0sN7A",
#       "status": "completed",
#       "role": "assistant",
#       "type": "message",
#       "content": [
#         {
#           "type": "output_text",
#           "text": "Done! The answer is **4**, and I've saved it to the answer.md file."
#         },
#         {
#           "type": "output_file",
#           "fileUrl": "https://private-us-east-1.manuscdn.com/sessionFile/.../answer.md?Policy=...&Key-Pair-Id=...&Signature=...",
#           "fileName": "answer.md",
#           "mimeType": "text/markdown"
#         }
#       ]
#     }
#   ],
#   "credit_usage": 22
# }
#    POST / -> 200 OK  (duration: 561.8 ms, execution: 465.0 ms)
```

## Conclusion

In this notebook, you learned how to move beyond the limitations of polling and create a more efficient, scalable solution for monitoring Manus tasks.

By using Modal and FastAPI, you deployed a serverless webhook receiver that can instantly process task events from Manus. You learned how to securely manage API credentials with Modal Secrets and how to fetch the full conversation history and generated files when a task completes.

Let's recap the journey so far:

1. In Notebook 1, you mastered the fundamentals of creating and managing asynchronous tasks with the Manus API.
2. In Notebook 2, you learned how to provide rich context to your tasks by attaching files, enabling far more complex and useful automations.
3. And now, in this notebook, you've built the crucial event-driven layer that allows your applications to react instantly and efficiently, completing the core toolkit for building sophisticated workflows.

In the next section, we'll walk through how to build a simple slack bot that ties all of this together - allowing you to use the Manus Agent to proactively investigate support tickets before you even look at them, providing a rich amount of context before you look at them.